In [10]:
import numpy as np

class AntColonyJobScheduling:
    def __init__(self, processing_times, n_ants=10, n_iterations=50, alpha=1, beta=2, evaporation_rate=0.5, q=100):

        self.processing_times = np.array(processing_times)
        self.n_jobs = len(processing_times)
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.q = q


        self.pheromone = np.ones((self.n_jobs, self.n_jobs))

        self.heuristic = 1 / (self.processing_times + 1e-6)

    def _select_next_job(self, current_job, unvisited):
        pheromone = self.pheromone[current_job][unvisited] ** self.alpha
        heuristic = self.heuristic[unvisited] ** self.beta
        probs = pheromone * heuristic
        probs /= probs.sum()
        next_job = np.random.choice(unvisited, p=probs)
        return next_job

    def _build_solution(self):
        solution = []
        unvisited = list(range(self.n_jobs))
        current_job = np.random.choice(unvisited)
        solution.append(current_job)
        unvisited.remove(current_job)

        while unvisited:
            next_job = self._select_next_job(current_job, unvisited)
            solution.append(next_job)
            unvisited.remove(next_job)
            current_job = next_job

        return solution

    def _calculate_makespan(self, solution, n_machines=2):

        machine_times = [0]*n_machines
        for job in solution:
            idx = np.argmin(machine_times)
            machine_times[idx] += self.processing_times[job]
        return max(machine_times)

    def _update_pheromone(self, solutions, makespans):
        self.pheromone *= (1 - self.evaporation_rate)
        for sol, mk in zip(solutions, makespans):
            for i in range(len(sol) - 1):
                self.pheromone[sol[i]][sol[i+1]] += self.q / mk

    def run(self, n_machines=2):
        best_solution = None
        best_makespan = float('inf')

        for iteration in range(self.n_iterations):
            solutions = []
            makespans = []
            for _ in range(self.n_ants):
                sol = self._build_solution()
                mk = self._calculate_makespan(sol, n_machines)
                solutions.append(sol)
                makespans.append(mk)
                if mk < best_makespan:
                    best_makespan = mk
                    best_solution = sol
            self._update_pheromone(solutions, makespans)
            print(f"Iteration {iteration+1}, Best Makespan: {best_makespan}")

        return best_solution, best_makespan

processing_times = [2, 14, 4, 16, 6, 5, 3]
aco = AntColonyJobScheduling(processing_times, n_ants=20, n_iterations=5)
best_schedule, best_time = aco.run(n_machines=3)
print("\nBest job order:", best_schedule)
print("Best makespan:", best_time)


Iteration 1, Best Makespan: 18
Iteration 2, Best Makespan: 18
Iteration 3, Best Makespan: 18
Iteration 4, Best Makespan: 17
Iteration 5, Best Makespan: 17

Best job order: [np.int64(3), np.int64(0), np.int64(6), np.int64(2), np.int64(1), np.int64(5), np.int64(4)]
Best makespan: 17
